# Tests Numpy

## Format JSON

In [1]:
import numpy as np
from pprint import pprint
from test_numpy import to_json_tab, to_json, from_json_tab, from_json
from cfield import Cutil
from dataset import Sdataset
import ntv_pandas as npd

a = np.arange(1,25).reshape((2,3,2,2))

print("example  :\n")
pprint(to_json(a), width=150)

axes = {'x': ['x1', 'x2'], 
        'y': ['y1', 'y2', 'y3'], 
        'z': ['z1', 'z2'], 
        'option': [True, False]}

print("\nexample with axes :\n")
pprint(to_json(a, axes), width=150)

example  :

[[[[[1, 2], [3, 4]], [[5, 6], [7, 8]], [[9, 10], [11, 12]]], [[[13, 14], [15, 16]], [[17, 18], [19, 20]], [[21, 22], [23, 24]]]], 'int32']

example with axes :

{':ndarray': [[[[[1, 2], [3, 4]], [[5, 6], [7, 8]], [[9, 10], [11, 12]]], [[[13, 14], [15, 16]], [[17, 18], [19, 20]], [[21, 22], [23, 24]]]],
              'int32'],
 'xnames': ['x', 'y', 'z', 'option'],
 'xvalues': [['x1', 'x2'], ['y1', 'y2', 'y3'], ['z1', 'z2'], [True, False]]}


In [2]:
print("reversibility :\n")
a2, axes2 = from_json(to_json(a, axes))
print(np.array_equal(a2, a), axes2 == axes)

reversibility :

True True


In [3]:
print("example with tabular format :\n")
pprint(to_json_tab(a, axes), width=150)

example with tabular format :

{'::int32': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 'option': [[True, False], [1]],
 'x': [['x1', 'x2'], [12]],
 'y': [['y1', 'y2', 'y3'], [4]],
 'z': [['z1', 'z2'], [2]]}


In [7]:
print("reversibility :\n")
a2, axes2 = from_json_tab(to_json_tab(a, axes))
print(np.array_equal(a2, a), axes2 == axes)
print(axes2)
print(axes)

reversibility :

[['x', None, ["x1", "x2"], None, None, 12, 24], ['y', None, ["y1", "y2", "y3"], None, None, 4, 12], ['z', None, ["z1", "z2"], None, None, 2, 4], ['option', None, [true, false], None, None, 1, 2], ['', 'int32', [{":int32": 1}, {":int32": 2}, {":int32": 3}, {":int32": 4}, {":int32": 5}, {":int32": 6}, {":int32": 7}, {":int32": 8}, {":int32": 9}, {":int32": 10}, {":int32": 11}, {":int32": 12}, {":int32": 13}, {":int32": 14}, {":int32": 15}, {":int32": 16}, {":int32": 17}, {":int32": 18}, {":int32": 19}, {":int32": 20}, {":int32": 21}, {":int32": 22}, {":int32": 23}, {":int32": 24}], None, None, None, 24]]
<class 'str'>
True False
{'x': ["x1", "x2"], 'y': ["y1", "y2", "y3"], 'z': ["z1", "z2"], 'option': [true, false]}
{'x': ['x1', 'x2'], 'y': ['y1', 'y2', 'y3'], 'z': ['z1', 'z2'], 'option': [True, False]}


Tabular format is compatible with tabular tools (e.g. Pandas).

In [5]:
print('pandas DataFrame :')
npd.read_json(to_json_tab(a, axes))

pandas DataFrame :


,x,y,z,option,None
0,x1,y1,z1,True,1
1,x1,y1,z1,False,2
2,x1,y1,z2,True,3
3,x1,y1,z2,False,4
4,x1,y2,z1,True,5
5,x1,y2,z1,False,6
6,x1,y2,z2,True,7
7,x1,y2,z2,False,8
8,x1,y3,z1,True,9
9,x1,y3,z1,False,10
